# Libraries

In [70]:
import pandas as pd
import numpy as np
import ast
import plotly.express as px
import re
from sklearn.preprocessing import MultiLabelBinarizer
from selenium import webdriver

# Import Data

In [22]:
booking = pd.read_csv('booking_all.csv')

In [23]:
booking.head(21)

,title,price,previous_price,location_score,property_type,address,score,score_label,n_reviews,description,appliances,popular_facilities,score_categories
0,SmartRental Collection Gran Vía Capital,€ 252,€ 420,Location 9.8,Condo Hotel,"14 Calle Gran Vía, Madrid City Center, 28013 M...",8.9,Excellent,"1,122 reviews",You're eligible for a Genius discount at Smart...,"Apartments,Kitchen,Swimming pool,Washing machi...","Electric vehicle charging station,Dining table...","{'Staff': '9.3', 'Facilities': '9.1', 'Cleanli..."
1,Apartments Center Madrid,€ 288,NaN,Location 9.4,Apartments,"Calle de los Jardines 16, Madrid City Center, ...",7.8,Good,648 reviews,You're eligible for a Genius discount at Apart...,"Apartments,Kitchen,Washing machine,Free WiFi,B...","Parking garage,Electric vehicle charging stati...","{'Staff': '7.4', 'Facilities': '7.9', 'Cleanli..."
2,Plaza Mayor Family,€ 312,€ 375,Location 9.9,Apartment,"calle Mayor 32 1 1, Madrid City Center, 28013 ...",8.8,Excellent,122 reviews,You're eligible for a Genius discount at Plaza...,"Whole apartment,80 m²\nSize,Kitchen,City view,...","Parking garage,Accessible parking,Dining table...","{'Staff': '9.5', 'Facilities': '8.6', 'Cleanli..."
3,Plaza España,€ 200,€ 208,NaN,Apartments,"Calle de los Reyes 11, Madrid City Center, 280...",7.1,Good,240 reviews,You're eligible for a Genius discount at Plaza...,"Apartments,Kitchen,Pet friendly,Washing machin...","Dining table,Coffee machine,Stovetop,Oven,Kitc...","{'Staff': '7.5', 'Facilities': '7.2', 'Cleanli..."
4,HABITACION PRIVADA EN PLENO CENTRO DE MADRID,€ 95,NaN,Location 10,Homestay,"13 Calle de la Ruda, 3ro C exterior, Madrid Ci...",10.0,Exceptional,1 review,"Located in Madrid, a 8-minute walk from Plaza ...",NaN,"Toilet paper,Towels,Bathtub or shower,Shared t...","{'Staff': '7.5', 'Facilities': '10', 'Cleanlin..."
5,Comfy 4 rooms apt in the heart of Chueca (Madr...,€ 311,NaN,Location 9.8,Apartment,"16 Calle de las Infantas, Madrid City Center, ...",8.8,Excellent,13 reviews,"Featuring a hot tub, Comfy 4 rooms apt in the ...","Whole apartment,135 m²\nSize,Kitchen,View,Pet ...","Oven,Kitchen,Washing machine,Microwave,Refrige...","{'Staff': '9.6', 'Facilities': '8.7', 'Cleanli..."
6,Parsa HomStay,€ 81,NaN,NaN,Guesthouse,"Plaza de Isabel II, Madrid City Center, 28013 ...",6.6,Pleasant,44 reviews,"Located in Madrid, Parsa HomStay provides air-...",NaN,"Toilet paper,Towels,Shared kitchen,Kitchenware...","{'Staff': '7.8', 'Facilities': '6.8', 'Cleanli..."
7,Axel Hotel Madrid - Adults Only,€ 258,NaN,Location 9.4,Hotel,"Atocha, 49, Madrid City Center, 28012 Madrid, ...",8.6,Excellent,"3,298 reviews","Set in Madrid city center, Axel Hotel Madrid -...",NaN,"Toilet paper,Towels,Slippers,Private Bathroom,...","{'Staff': '9.2', 'Facilities': '8.7', 'Cleanli..."
8,Riu Plaza España,€ 288,€ 443,Location 9.7,Hotel,"Calle Gran Vía, 84, Madrid City Center, 28013 ...",8.9,Excellent,"28,112 reviews","Well located in the center of Madrid, Riu Plaz...","Swimming pool,Free WiFi,Air conditioning,Priva...","Toilet paper,Towels,Private Bathroom,Toilet,Fr...","{'Staff': '9.3', 'Facilities': '9.1', 'Cleanli..."
9,Room Mate Oscar,€ 324,NaN,Location 9.7,Hotel,"Plaza de Pedro Zerolo, 12, Madrid City Center,...",8.8,Excellent,"2,703 reviews",You're eligible for a Genius discount at Room ...,"Swimming pool,Free WiFi,Air conditioning,Priva...","Toilet paper,Towels,Guest bathroom,Slippers,Pr...","{'Staff': '9.5', 'Facilities': '8.9', 'Cleanli..."


# Cleaning

In [24]:
#remove na values
booking[~booking['price'].isna()].shape[0]/booking.shape[0]
booking_clean = booking[~booking['price'].isna()]

In [25]:
booking_clean['price_int'] = booking_clean['price'].str.replace('€','')
booking_clean['price_int'] = booking_clean['price_int'].str.replace(',','')
booking_clean['price_int'] = booking_clean['price_int'].astype(int)
booking_clean['previous_price_int'] = booking_clean['previous_price'].str.replace('€','').str.replace(',','').astype(float)
booking_clean['location_score_float'] = booking_clean['location_score'].str.replace('Location','').astype(float)
booking_clean['score_float'] = booking_clean['score'].astype(float)
booking_clean['n_reviews_float'] = booking_clean['n_reviews'].str.extract('(\d*,*\d+)')
booking_clean['n_reviews_float'] = booking_clean['n_reviews_float'].str.replace(',','').astype(float)

C:\Users\antoi\AppData\Local\Temp\ipykernel_15600\3596275480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_clean['price_int'] = booking_clean['price'].str.replace('€','')
C:\Users\antoi\AppData\Local\Temp\ipykernel_15600\3596275480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_clean['price_int'] = booking_clean['price_int'].str.replace(',','')
C:\Users\antoi\AppData\Local\Temp\ipykernel_15600\3596275480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [26]:
booking_clean = pd.get_dummies(booking_clean,columns=['property_type','score_label'],prefix = ['property','score'])
booking_clean[['property_type','score_label']] = booking[['property_type','score_label']]


In [27]:
booking_clean['appliances_list'] = booking_clean['appliances'].str.split(',')
appliances_onehot = pd.get_dummies(booking_clean['appliances_list'].apply(pd.Series).stack(dropna=False),prefix = 'appliances',dummy_na = True).groupby(level=0).sum()
booking_clean = pd.concat([booking_clean,appliances_onehot],axis = 1)

# Visualisations

In [121]:
booking_clean[['property_type','price_int']].groupby('property_type').mean().sort_values(by = 'price_int').reset_index()

,property_type,price_int
0,Capsule Hotel,93.660377
1,Homestay,119.843137
2,Hostel,143.105145
3,Guesthouse,144.192241
4,Apartments,229.476476
5,Condo Hotel,241.061611
6,Bed and Breakfast,267.750000
7,Apartment,268.628571
8,Hotel,307.091803


In [117]:
# Note moyenne par 
def plot_avg(data,groupcol,aggcol,min = 0,max = None,width = 800,height = 400,title = 'title'):
    if max == None:
       max =  np.unique(data[groupcol]).shape[0] -1

    df_avg = data[[groupcol,aggcol]].groupby(groupcol).mean().sort_values(by = aggcol).reset_index()[min:max]


    fig = px.bar(df_avg,x = aggcol,y = groupcol,orientation='h',width = width,height=height,title=title)
   #  fig.update_layout(yaxis = dict(tickmode = 'linear'))
    return fig.show()

In [66]:
def plot_counts(data,col,min = 0,max = None,width = 800,height = 400,title = 'title'):
    if max == None:
       max =  np.unique(data[col]).shape[0] -1

    df_counts = pd.DataFrame(data[col].value_counts()).reset_index().rename(columns = {col:f'{col}_counts','index':col}).sort_values(by = f'{col}_counts')
    fig = px.bar(df_counts,x = f'{col}_counts',y=col,orientation='h',width = width,height=height,title=title)
    fig.update_layout(yaxis = dict(tickmode = 'linear'))
    return fig.show()

In [114]:
def plot_scatter(data,col1,col2,col3 = None,x_limit = None,y_limit=None,names = None,width = 800,height = 400,title = 'title',
col1_name = None,col2_name = None,col3_name = None):
    df_scatter = data[[col1,col2,col3]]
    if x_limit != None:
        df_scatter = df_scatter[df_scatter[col1]<=x_limit]
    if y_limit != None:
        df_scatter = df_scatter[df_scatter[col2]<=y_limit]
    if names != None:
        df_scatter = df_scatter[df_scatter[col3].isin(names)]
    
    if col1_name == None:
        col1_name = col1
    if col2_name == None:
        col2_name = col2
    if col3_name == None:
        col3_name = col3

    fig = px.scatter(df_scatter,col1,col2,color = col3,width=width,height = height,title = title,
    labels={col1:col1_name, col2:col2_name, col3:col3_name})
    return fig.show()

In [56]:
plot_avg(booking_clean,'property_type','price_int',width = 800,height=400,title='Average price per type of stay')

In [57]:
plot_avg(booking_clean,'property_type','location_score_float',width = 800,height=400,title='Average location score per stay')

In [67]:
plot_counts(booking_clean,'property_type',title='Number of property types')

In [113]:
plot_scatter(booking_clean,'price_int','score_float','property_type',x_limit = 1000,names=['Guesthouse','Hotel','Apartment'],
title = 'Notes en fonction du prix et du type de sejour',col1_name='prix',col2_name='note',col3_name='type')

prix


In [118]:
plot_avg(booking_clean,'property_type','n_reviews_float',title = 'Average number of reviews per type')

In [120]:
plot_scatter(booking_clean,'n_reviews_float','score_float','property_type',x_limit = 5000,names=['Guesthouse','Hotel','Apartment'],
title = 'Notes en fonction du prix et du type de sejour',col1_name='number of reviews',col2_name='note',col3_name='type')